# Trouver le fichier à modifier

In [1]:
import ast
import os
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm

##### Creation des modèle

- [CodeT5+](https://github.com/salesforce/CodeT5) sert à traduire une fonction en une phrase qui la résume ( la fonction ne doit pas faire plus de 512 tokens)
- Bert ([MiniLM](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)) sert à mettre dans un embedding le texte à comparer et le code résumé

Modifier la variable `device` par 'cuda' pour utiliser le GPU ce qui augmenterai les performance du modèle.

In [2]:
checkpoint = "Salesforce/codet5p-220m-bimodal"
device = "cpu"  

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
codeT5 = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)

bert = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



## Récupération des fonctions

Ces deux blocs de codes ci-dessous servent à isoler et récupérer les fonctions qui se trouvent dans un fichier de code.\
Pour ce faire, nous utilisons des AST, mais cette fonction ne fonctionne sur des fichiers Python. \
Donc si on veut s'ouvrir à d'autres langages il faut trouver soit une fonction AST plus complète ou une autre possibilité serai d'utiliser des Regex

In [3]:
def extract_function_source(node):
    if isinstance(node, ast.FunctionDef):
        return ast.unparse(node)

In [4]:
functions_sources = []
for filename in tqdm(os.listdir('samples/')):
    if os.path.isfile(os.path.join('samples/', filename)):
        with open('samples/' + filename, "r") as file:
            file_content = file.read()
        
        parsed_tree = ast.parse(file_content)
        

        functions = [extract_function_source(node) for node in ast.walk(parsed_tree) if isinstance(node, ast.FunctionDef)]
        if functions :
            for function in functions:
                functions_sources.append([
                    'samples/' + filename,
                    function,
                ])

    

100%|██████████| 15/15 [00:00<00:00, 316.29it/s]


## Résumé

Cette partie permet de transformer le code en texte (anglais)

In [5]:
for function_source in tqdm(functions_sources):
    input_ids = tokenizer(function_source[1], return_tensors="pt").input_ids.to(device)


    generated_ids = codeT5.generate(input_ids, max_length=20)
    function_source.append(tokenizer.decode(generated_ids[0], skip_special_tokens=True))



100%|██████████| 136/136 [01:16<00:00,  1.77it/s]


## Calcul de similitude

Dans la variable text est la phrase remontant l'issue, l'anomalie...\
Dans la suite du bloc nous comparons la similitude entre le texte et la fonction afin de trouver le fichier qu'il faudra potentiellement modifier.\
Nous utilisons la similarité cosinus pour faire la comparaison.


In [6]:
text = 'I have a problem with the colisions of my block'
file = ''
max_similitude = float('-inf')

for function_source in tqdm(functions_sources):
    sentences = [text, function_source[2]]

    #Compute embedding for both lists
    embedding_1= bert.encode(sentences[0], convert_to_tensor=True)
    embedding_2 = bert.encode(sentences[1], convert_to_tensor=True)

    if max_similitude < util.pytorch_cos_sim(embedding_1, embedding_2):
        max_similitude = util.pytorch_cos_sim(embedding_1, embedding_2)
        file = function_source[0]

print(f"Le fichier qui match le plus est {file} avec une sim cos de {max_similitude}")

100%|██████████| 136/136 [00:03<00:00, 42.08it/s]

Le fichier qui match le plus est samples/tetris.py avec une sim cos de tensor([[0.6263]])
